# Oscillator Comparison: Theory vs. Practice

This notebook compares three implementations of a sine wave oscillator:
- **Wavetable lookup**: pre-compute the waveform once, keep it in a Python list, and then look up values
- **Direct sin()**: Calculate sin() from scratch for every sample
- **NumPy wavetable**: Vectorize array operations to improve performance

In Python with lists, `sin()` is faster than a wavetable, because it's optimized C code. This is an example of how big O's theoretical complexity doesn't account for hardware differences: both wavetable and direct `sin()` solutions are O(1).

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
import time

# Make plots look nice
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Oscillator Functions

In [2]:
def wavetable_oscillator(num_samples, frequency=1.0, sample_rate=44100, table_size=1024):
    """Generate Oscillator using pre-computed wavetable"""
    # Create one cycle - frequency determines playback speed, not table contents
    table = [math.sin(2 * math.pi * i / table_size) for i in range(table_size)]
    
    output = []
    phase = 0.0
    phase_increment = frequency / sample_rate
    
    for _ in range(num_samples):
        # Find exact position in table
        exact_table_position = phase * table_size
        index_before = int(exact_table_position)
        index_after = (index_before + 1) % table_size
        blend_amount = exact_table_position - index_before
        
        # Interpolate between two nearest samples
        sample_before = table[index_before]
        sample_after = table[index_after]
        output_value = sample_before + blend_amount * (sample_after - sample_before)
        
        output.append(output_value)
        
        # Advance phase
        phase += phase_increment
        if phase >= 1.0:
            phase -= 1.0
    
    return output


def direct_sin_oscillator(num_samples, frequency=1.0, sample_rate=44100):
    """Generate Oscillator by calculating sin() each time"""
    output = []
    phase = 0.0
    phase_increment = frequency / sample_rate
    
    for _ in range(num_samples):
        # Calculate sin directly
        output_value = math.sin(2 * math.pi * phase)
        output.append(output_value)
        
        # Advance phase
        phase += phase_increment
        if phase >= 1.0:
            phase -= 1.0
    
    return output


def numpy_wavetable_oscillator(num_samples, frequency=1.0, sample_rate=44100, table_size=1024):
    """Generate Oscillator using NumPy vectorized operations - the FAST way"""
    # Pre-compute table
    table = np.sin(2 * np.pi * np.arange(table_size) / table_size)
    
    # Generate all phase values at once (vectorized!)
    phase_increment = frequency / sample_rate
    phases = np.arange(num_samples) * phase_increment
    phases = phases % 1.0  # Wrap phases
    
    # Convert phases to table positions
    exact_positions = phases * table_size
    indices_before = exact_positions.astype(int)
    indices_after = (indices_before + 1) % table_size
    blend_amounts = exact_positions - indices_before
    
    # Interpolate (all at once!)
    samples_before = table[indices_before]
    samples_after = table[indices_after]
    output = samples_before + blend_amounts * (samples_after - samples_before)
    
    return output

## Performance Comparison

Let's measure the actual performance differences:

In [5]:
@interact(
    duration=IntSlider(min=1, max=20, step=1, value=10, description='Test Duration (sec)')
)
def benchmark_oscillator(duration=10):
    sample_rate = 44100
    num_samples = sample_rate * duration
    frequency = 1.0
    
    # Benchmark Python list wavetable
    start = time.time()
    wavetable_result = wavetable_oscillator(num_samples, frequency=frequency, sample_rate=sample_rate)
    wavetable_time = time.time() - start
    
    # Benchmark direct sin
    start = time.time()
    sin_result = direct_sin_oscillator(num_samples, frequency=frequency, sample_rate=sample_rate)
    sin_time = time.time() - start
    
    # Benchmark NumPy wavetable
    start = time.time()
    numpy_result = numpy_wavetable_oscillator(num_samples, frequency=frequency, sample_rate=sample_rate)
    numpy_time = time.time() - start
    
    # Create bar chart
    fig, ax = plt.subplots(figsize=(12, 7))
    methods = ['Wavetable\n(Python lists)', 'Direct sin()\n(math.sin)', 'NumPy Wavetable\n(vectorized)']
    times = [wavetable_time, sin_time, numpy_time]
    colors = ['#e74c3c', '#f39c12', '#2ecc71']
    
    bars = ax.bar(methods, times, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    ax.set_ylabel('Time (seconds)', fontsize=12)
    ax.set_title(f'Performance Comparison - {duration} seconds of audio @ 44.1kHz', 
                 fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar, time_val in zip(bars, times):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{time_val:.4f}s',
                ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Calculate speedups
    sin_vs_wavetable = wavetable_time / sin_time
    numpy_vs_sin = sin_time / numpy_time
    numpy_vs_wavetable = wavetable_time / numpy_time
    
    # Add speedup text
    speedup_text = (
        f'🔴 Python wavetable is {sin_vs_wavetable:.1f}x SLOWER than sin()!\n'
        f'🟢 NumPy wavetable is {numpy_vs_sin:.1f}x faster than sin()\n'
        f'🟢 NumPy wavetable is {numpy_vs_wavetable:.1f}x faster than Python wavetable'
    )
    ax.text(0.5, 0.97, speedup_text, 
            transform=ax.transAxes, fontsize=11, fontweight='bold',
            ha='center', va='top',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9))
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nResults for {duration} seconds of audio:")
    print(f"  Wavetable (Python lists): {wavetable_time:.4f} seconds")
    print(f"  Direct sin():             {sin_time:.4f} seconds")
    print(f"  NumPy wavetable:          {numpy_time:.4f} seconds")
    print(f"\nSpeedups:")
    print(f"  sin() vs Python wavetable:  {sin_vs_wavetable:.2f}x faster")
    print(f"  NumPy vs sin():             {numpy_vs_sin:.2f}x faster")
    print(f"  NumPy vs Python wavetable:  {numpy_vs_wavetable:.2f}x faster")
    print(f"\nTotal samples generated: {num_samples:,}")

interactive(children=(IntSlider(value=10, description='Test Duration (sec)', max=20, min=1), Output(outputs=({…

### Big O Doesn't Tell the Whole Story

Both of these approaches are **O(1)** per sample.

Big O ignores constant factors, but constant factors matter in real-world performance.

And implementation details can make "theoretically slower" code run faster.

### Time-Space Tradeoff
Note that the wavetable uses ~4KB memory (1024 floats) to store the lookup table, while the direct `sin()` implementation uses almost no memory. In 2024, 4KB is trivial, so memory cost isn't a real concern, and the tradeoff is more about implementation complexity vs. performance.

### In Conclusion
Don't trust theory alone. Benchmark (measure) your code's performance. Implementation matters as much as algorithm choice.